In [68]:
import itertools as it
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import functools
import math
import fractions
from scipy.interpolate import lagrange
from numpy import linalg as LA

In [59]:
def KronDel(x, y):
    if x == y:
        return 1.
    else:
        return 0.

In [60]:
def CanonicalMPI(func, x, k, say = 0):
    for i in range(k):
        if(say != 0):
            print (i, "th iteration, x is ", x)
        x = func(x)
    if(say != 0):
            print (k, "th iteration, x is ", x)
    return x

$\textbf{Метод отрезков}$

In [21]:
#g - функция выбора - пара точек на вход, пара на выходе
def sections(g, f, a, b, k, say = 0):
    I = np.array([a, b])
    for i in range(k):
        if(say != 0):
            print(i, "th iteration, x1 = ", I[0], ", x2 = ", I[1], " |x1 - x2| = ", np.abs(I[0] - I[1]))
        c = g(I)[0]
        d = g(I)[1]
        if (f(c) > f(d)):
            I = np.array([c, b])
            a = c
        else:
            I = np.array([a, d])
            b = d
    if(say != 0):
            print(k, "th iteration, x1 = ", I[0], ", x2 = ", I[1], " |x1 - x2| = ", np.abs(I[0] - I[1]))
    return I[0]

In [50]:
#Пример:
def g(I):
    return np.array([I[0] + (1/3)*(I[1] - I[0]), I[0] + (2/3)*(I[1] - I[0])])
def f1(x):
    return x**2
sections(g, f1, -1., 1., 25, 0)

-3.4143342277530024e-05

$\textbf{Метод золотого сечения}$

In [32]:
def coolg(I):
    return np.array([I[0] + (I[1] - I[0])*((3-(5**0.5))/2), I[1] - (I[1] - I[0])*((3-(5**0.5))/2)])   

In [57]:
#Пример:
sections(coolg, f1, -1., 1., 25, 0)

-9.644875678381916e-06

$\textbf{Метод парабол}$

In [47]:
def Par(f, h, x0, k, say):
    def temp(y):
        return y - (h/2)*(f(y + h) - f(y - h))/(f(y + h) + f(y - h) - 2*f(y))
    return CanonicalMPI(temp, x0, k, say)

$\textbf{Метод покоординатного спуска:}$

In [75]:
#Пусть g(f0) - значение, минимизирующее f0 (f0 - ф-я одной переменной)
def coord(g, f, I0, k, say = 0):
    for l in range(k):
        for i in range(len(I0)):
            if(say != 0):
                print(l, "th iteration, ", i, "th coordinate:", I0)
            def tmp(y):
                J = np.array([I0[j]*(1 - KronDel(i,j)) + KronDel(i,j)*y for j in range(len(I0))])
                return f(J)
            I0[i] = g(tmp)
        if(say != 0):
                print(l, "th iteration, ", len(I0), "th coordinate:", I0)
    return I0

In [110]:
#Пример
def G(F):
    return Par(F, 0.01, 1, 100, 0)
def f(I):
    return I[0]**2 + (I[1] - I[0] - 1)**2
coord(G, f, np.array([2., 3.]), 10, 1)

0 th iteration,  0 th coordinate: [2. 3.]
0 th iteration,  1 th coordinate: [1. 3.]
0 th iteration,  2 th coordinate: [1. 2.]
1 th iteration,  0 th coordinate: [1. 2.]
1 th iteration,  1 th coordinate: [0.5 2. ]
1 th iteration,  2 th coordinate: [0.5 1.5]
2 th iteration,  0 th coordinate: [0.5 1.5]
2 th iteration,  1 th coordinate: [0.25 1.5 ]
2 th iteration,  2 th coordinate: [0.25 1.25]
3 th iteration,  0 th coordinate: [0.25 1.25]
3 th iteration,  1 th coordinate: [0.125 1.25 ]
3 th iteration,  2 th coordinate: [0.125 1.125]
4 th iteration,  0 th coordinate: [0.125 1.125]
4 th iteration,  1 th coordinate: [0.0625 1.125 ]
4 th iteration,  2 th coordinate: [0.0625 1.0625]
5 th iteration,  0 th coordinate: [0.0625 1.0625]
5 th iteration,  1 th coordinate: [0.03125 1.0625 ]
5 th iteration,  2 th coordinate: [0.03125 1.03125]
6 th iteration,  0 th coordinate: [0.03125 1.03125]
6 th iteration,  1 th coordinate: [0.015625 1.03125 ]
6 th iteration,  2 th coordinate: [0.015625 1.015625]
7 th

array([0.00195313, 1.00195312])

$\textbf{Метод градиентного спуска}$

Идея:
Итерируемся сдвигая x вдоль прямой на которой f больше всего убывает (против градиента) так чтобы на этой прямой f была минимальна.

In [115]:
def grad(g, f, df, I0, k, say = 0):
    for i in range(k):
        if(say != 0):
            print(i,"th iteration", "x = ", I0)
        def temp(y):
            return f(I0 - y*df(I0))
        I0 = (I0 - g(temp)*df(I0))
    if(say != 0):
        print(k,"th iteration", "x = ", I0)
    return I0

In [116]:
#Пример
def dF(L):
    return np.array([2*L[0] - 2*(L[1] - L[0] - 1), 2*(L[1] - L[0] - 1)])
grad(G, f, dF,np.array([2., 3.]), 100, 1)
dF([2.,3.])

0 th iteration x =  [2. 3.]
1 th iteration x =  [1. 3.]
2 th iteration x =  [1. 2.]
3 th iteration x =  [0.5 2. ]
4 th iteration x =  [0.5 1.5]
5 th iteration x =  [0.25 1.5 ]
6 th iteration x =  [0.25 1.25]
7 th iteration x =  [0.125 1.25 ]
8 th iteration x =  [0.125 1.125]
9 th iteration x =  [0.0625 1.125 ]
10 th iteration x =  [0.0625 1.0625]
11 th iteration x =  [0.03125 1.0625 ]
12 th iteration x =  [0.03125 1.03125]
13 th iteration x =  [0.015625 1.03125 ]
14 th iteration x =  [0.015625 1.015625]
15 th iteration x =  [0.0078125 1.015625 ]
16 th iteration x =  [0.0078125 1.0078125]
17 th iteration x =  [0.00390625 1.0078125 ]
18 th iteration x =  [0.00390625 1.00390625]
19 th iteration x =  [0.00195312 1.00390625]
20 th iteration x =  [0.00195312 1.00195312]
21 th iteration x =  [9.76562500e-04 1.00195312e+00]
22 th iteration x =  [9.76562500e-04 1.00097656e+00]
23 th iteration x =  [4.88281250e-04 1.00097656e+00]
24 th iteration x =  [4.88281250e-04 1.00048828e+00]
25 th iterati

array([4., 0.])